# Topic modelling of news headlines for prediction of news category with 6 topics

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier on the combined feature 'title_description_text' feature and assume 6 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import pyLDAvis.lda_model

import nlp_preprocessors

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\certifi\core.py:36: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  _CACERT_CTX = get_path("certifi", "cacert.pem")


## Load data

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Clean text

In [7]:
train['ti_de_te_cleaned'] = train['title_description_text'].apply(lambda x: nlp_preprocessors.clean_text(x))

## Extract features from 'title_description_text' 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words=stop_words)

In [10]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['ti_de_te_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [11]:
dtm

<67513x29994 sparse matrix of type '<class 'numpy.int64'>'
	with 1026120 stored elements in Compressed Sparse Row format>

In [12]:
from sklearn.decomposition import LatentDirichletAllocation

In [13]:
lda = LatentDirichletAllocation(n_components=6, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=6, random_state=42)

## Analyse extracted features

In [14]:
len(cv.get_feature_names_out())

29994

In [15]:
cv.get_feature_names_out()

array(['000', '01', '034', ..., 'đoković', 'войны', 'украинцы'],
      dtype=object)

In [16]:
len(lda.components_)

6

In [17]:
lda.components_

array([[14.00755783,  0.16666719,  5.18607024, ...,  0.16666702,
         0.16667001,  0.16666999],
       [23.52251668,  0.16671058,  9.86061821, ...,  0.16673405,
         4.16665146,  0.16667006],
       [ 2.19246081,  0.16666716, 10.23530109, ...,  0.16666701,
         0.16666983,  0.16666981],
       [ 0.16994032,  0.70156754, 29.45859678, ...,  0.16677223,
         0.16666901,  0.16666899],
       [ 4.46161208,  3.63118249, 35.09225977, ...,  3.16624056,
         0.16666933,  5.16665082],
       [16.64591227,  0.16720504,  0.16715391, ...,  0.16691914,
         0.16667036,  0.16667033]])

In [18]:
len(lda.components_[0])

29994

### Show most important words of first extracted topic

In [19]:
first_topic = lda.components_[0]

In [20]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([25046, 17447, 28947, ..., 11666, 16450,  5532], dtype=int64)

In [21]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [22]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

deutsch
zahl
euro
gut
groß
steigen
geben
hoch
mensch
deutschland


### Show most important words of first extracted topic

In [23]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['million', 'inflation', 'preis', 'britisch', 'lindner', 'deutsch', 'zahl', 'euro', 'gut', 'groß', 'steigen', 'geben', 'hoch', 'mensch', 'deutschland']


Die TOP-15 Wörter für das Thema #1
['gut', 'experte', 'online', 'zeigen', 'taiwan', 'geben', 'groß', 'putin', 'land', 'euro', 'deutschland', 'usa', 'deutsch', 'us', 'china']


Die TOP-15 Wörter für das Thema #2
['weit', 'donald', 'töten', 'geben', 'mann', 'tote', 'gericht', 'protest', 'präsident', 'polizei', 'ex', 'sterben', 'trump', 'mensch', 'us']


Die TOP-15 Wörter für das Thema #3
['entwicklung', 'kanzler', 'habeck', 'moskau', 'selenskyj', 'kiew', 'eu', 'präsident', 'scholz', 'ukrainisch', 'putin', 'russisch', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #4
['stehen', 'gut', 'lauterbach', 'deutschland', 'geben', 'partei', 'wahl', 'grüne', 'spd', 'berlin', 'afd', 'online', 'focus', 'cdu', 'corona']


Die TOP-15 Wörter für das Thema #5
['fc', 'iran', 'bayern', 'land', 'generat

### Get topics with highest probability for news items in train data

In [24]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 6)

In [25]:
topic_results[0].round(5)

array([0.36251, 0.18005, 0.00728, 0.12336, 0.31954, 0.00726])

In [26]:
topic_results[0].argmax()

0

In [27]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [28]:
train.head()

,date,title,description,category,text,source,title_description_text,ti_de_te_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,0
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,3
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,4
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben rund Mädch...,2
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,5


## Visualize model

In [29]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=                x          y  topics  cluster       Freq
topic                                                   
3      -56.400227  93.653191       1        1  21.400051
4     -116.331902  -3.196838       2        1  19.204321
2      -19.901344   3.972522       3        1  15.807424
0      -43.252079 -89.995522       4        1  15.403958
1       61.954079 -47.543461       5        1  14.672082
5       54.253056  65.924736       6        1  13.512165, topic_info=            Term          Freq         Total Category  logprob  loglift
25499    ukraine  11628.000000  11628.000000  Default  30.0000  30.0000
14352      krieg   5856.000000   5856.000000  Default  29.0000  29.0000
21153   russland   5908.000000   5908.000000  Default  28.0000  28.0000
21147   russisch   5571.000000   5571.000000  Default  27.0000  27.0000
4946       china   2851.000000   2851.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
14878       land    683.683064   3810.897311   Topic6  -5.4410   0.2835
7470      erster    566.221743   2361.894822   Topic6  -5.6295   0.5733
23685     stehen    531.171660   2903.405454   Topic6  -5.6934   0.3030
20431  regierung    510.461911   2436.751760   Topic6  -5.7332   0.4385
9305       geben    533.186183   5117.002986   Topic6  -5.6896  -0.2599

[435 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
83         5  0.994829           49
627        2  0.999162          afd
631        2  0.014967  affenpocken
631        5  0.980313  affenpocken
643        3  0.048140  afghanistan
...      ...       ...          ...
29061      2  0.204147       zeigen
29061      3  0.116325       zeigen
29061      4  0.251139       zeigen
29061      5  0.246132       zeigen
29061      6  0.064325       zeigen

[901 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 1, 2, 6])

## Save train set with topics and dump countvectorizer and model

In [30]:
#Save result to csv
train.to_csv('train_lda_combined.csv')

In [31]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv_combined.jl')

['models/cv_combined.jl']

In [32]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_6_topics_combined.jl')

['models/lda_model_6_topics_combined.jl']